<a href="https://colab.research.google.com/github/arabs-in-neuro/intro_to_comp_neuro/blob/main/projects/fmri/brain_decoding_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain Decoding Tutorial
Mohamed Abdelhack

In order to understand the way the brain encodes sensory information, we need to find a relation between the stimulus and the measured brain response. The brain is always active as it handles a plethora of information related to different functions both internal and external.

In this tutorial, we will deal with data measured during visual stimulation experiments and investigate areas in the brain that respond to these visual images.

In [ ]:
# @markdown [Prep] Run this cell to load data and install libraries
# data download and library installation
# Generic object decoding data
!wget -q -O data.h5 https://ndownloader.figshare.com/files/15049646
# Generic object decoding features
!wget -q -O features.h5 https://ndownloader.figshare.com/files/15015971
# Download ImageNet labels
!wget -q -O imagenet_classes.txt https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

!pip install -q bdpy
!pip install -q seaborn_image

In [ ]:
# @markdown Include required libraries
import bdpy
from bdpy.util import get_refdata
import numpy as np
import pandas as pd
import seaborn_image as isns
import seaborn as sns
import ipywidgets as widgets  # interactive display
import matplotlib.pyplot as plt
from random import seed
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, cross_val_score
import urllib
from PIL import Image
from torchvision import transforms
import torch

In [ ]:
rois = {'VC' : 'ROI_VC = 1',
        'LVC' : 'ROI_LVC = 1',
        'HVC' : 'ROI_HVC = 1',
        'V1' : 'ROI_V1 = 1',
        'V2' : 'ROI_V2 = 1',
        'V3' : 'ROI_V3 = 1',
        'V4' : 'ROI_V4 = 1',
        'LOC' : 'ROI_LOC = 1',
        'FFA' : 'ROI_FFA = 1',
        'PPA' : 'ROI_PPA = 1'}

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_id)
    seed(worker_id)

## Part 1: Classification Models

### Generalized Linear Models (GLM)


#### Linear Regression

In [ ]:
# Simulate linear dependency

# Generate 1000 points randomly
x1 = np.random.normal(size=1000)
noise = np.random.normal(scale=0.3, size=1000)
y1 = 1.5 * x1
x1 = x1 + noise
sns.scatterplot(x=x1, y=y1)
plt.axis('equal')

In [ ]:
my_layout = widgets.Layout()

# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    slope=widgets.FloatSlider(0., min=-3., max=3., step=.1,
                               layout=my_layout),
    offset=widgets.FloatSlider(0., min=-3., max=3., step=0.2,
                                layout=my_layout)
)

def diff_DC(slope=200., offset=10.):
  fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,5))
  sns.scatterplot(x=x1, y=y1, ax=axes[0])
  x_data = np.array([-3, 3])

  y_data = x_data * slope + offset
  sns.lineplot(x=x_data, y=y_data, ax=axes[0], color='k')
  axes[0].axis('equal')
  axes[0].set_xlim(-3, 3)
  axes[0].set_ylim(-3, 3)

  error = np.square(y1 - (x1 * slope + offset))
  axes[1].stem(x1, error, use_line_collection=True)
  axes[1].set_ylim(0, 5)
  plt.show()

#### Logistic Regression

In [ ]:
# Logistic regression
x1 = np.random.normal(size=1000)
# noise = np.random.normal(scale=0.3, size=1000)
y1 = 1.5 * x1
y1_binary = (y1 > 0.0).astype(float)

my_layout = widgets.Layout()

def BinaryCrossEntropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term_0 = (1-y_true) * np.log(1-y_pred + 1e-7)
    term_1 = y_true * np.log(y_pred + 1e-7)
    return -np.mean(term_0+term_1, axis=0)

# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    slope=widgets.FloatSlider(1., min=0., max=10., step=.1,
                               layout=my_layout),
    offset=widgets.FloatSlider(0., min=-3., max=3., step=0.2,
                                layout=my_layout)
)



def diff_DC(slope=0., offset=0.):
  fig, axes = plt.subplots(1, 2, sharex=False, figsize=(10,5))
  sns.scatterplot(x=x1, y=y1_binary, ax=axes[0])
  x_data = np.linspace(-3, 3, 1000)

  y_data = 1 / (1 + np.exp(-(x_data * slope + offset)))

  sns.lineplot(x=x_data, y=y_data, ax=axes[0], color='k')
  axes[0].axis('equal')
  axes[0].set_xlim(-3, 3)
  axes[0].set_ylim(-0.1, 1.1)

  error = BinaryCrossEntropy(y1_binary, 1 / (1 + np.exp(-(x1 * slope + offset))))
  sns.barplot(x=[0], y=[error], ax=axes[1])
  axes[1].set_ylim(-0.1, 1.1)
  plt.show()


By moving the sliders, you are modifying the parameters of the linear regression model to produce the model that most likely has generated the data. This case of one input and one output is trivial but usually you will get multiple input variables for the output with some of the inputs related to the output to different degrees and some not related at all. The question here is how to tune your model to have the correct attributions between inputs and outputs and how to automate the process. One way to do that is to find the error between the real output features and the predicted output features of the model and try to minimize that. This loss is called mean squared error loss and is used predominantly in linear regression:

$$\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} \left( y_i - \hat{y}_i \right)^2$$


While for logistic regression, binary cross entropy function is used instead:


$$\text{BCE} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]$$


Minimizing the loss function can be achieved by multiple ways such as gradient descent. This is readily implemented in scikit learn for you as follows.

In [ ]:
linear_model = LinearRegression()
linear_model.fit(np.expand_dims(x1, axis=1), y1)
print(linear_model.coef_)

You can see the predicted weight is very close to the real one. At the end of the notebook, there is an exercise if you are interested in how this actually works on the inside.

Now let’s consider a case of fewer points and three inputs. You can now tune four parameters. In this condition, we want to test the quality of our model so we hide some points (test set) and you should tune the parameters on the visible points only (training set). First, imaging that the output is only dependent on x1 and observe the model.

In [ ]:
seed_worker(0)

x1 = np.random.normal(size=10)
noise1 = np.random.normal(scale=1, size=10)
x2 = np.random.normal(size=10)
noise2 = np.random.normal(scale=1, size=10)
x3 = np.random.normal(size=10, scale=1)
noise3 = np.random.normal(scale=1, size=10)
y1 = 0.9 * x1 + 0.4 * x2
x1 = x1 + noise1
x2 = x2 + noise2
x3 = x3 + noise3

In [ ]:
my_layout = widgets.Layout()

# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    beta0=widgets.FloatSlider(0., min=-4., max=4., step=.1,
                               layout=my_layout),
    beta1=widgets.FloatSlider(0., min=-4., max=4., step=.1,
                               layout=my_layout),
    beta2=widgets.FloatSlider(0., min=-4., max=4., step=.1,
                               layout=my_layout),
    beta3=widgets.FloatSlider(0., min=-4., max=4., step=.1,
                                layout=my_layout),
    proportion_train=widgets.FloatSlider(0.3, min=0., max=100., step=10.,
                                layout=my_layout),
    flag_m=widgets.Checkbox(value=False, description='show_test', disabled=False, indent=False)
)

def diff_DC(beta0=0., beta1=0., beta2=0.,
            beta3=0., proportion_train=0.5, flag_m=False):
  fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,5))
  number_of_train_points = int(proportion_train * x1.shape[0])
  sns.scatterplot(x=x1[:number_of_train_points],
                  y=y1[:number_of_train_points],
                  ax=axes[0], color='r')
  if flag_m:
    sns.scatterplot(x=x1[number_of_train_points:],
                  y=y1[number_of_train_points:],
                  ax=axes[0], color='g')
  x_data = x1

  y_data = x1 * beta1 + x2 * beta2 + x3 * beta3 + beta0
  sns.lineplot(x=x_data, y=y_data, ax=axes[0], color='k')
  axes[0].axis('equal')
  axes[0].set_xlim(-3, 3)
  axes[0].set_ylim(-3, 3)

  error = np.abs(y1 - y_data)
  axes[1].stem(x1[:number_of_train_points], error[:number_of_train_points], linefmt='red', markerfmt='ro', use_line_collection=True)
  if flag_m:
    axes[1].stem(x1[number_of_train_points:], error[number_of_train_points:], linefmt='green', markerfmt='go', use_line_collection=True)
  axes[1].set_ylim(0, 5)
  plt.show()

In [ ]:
# Prepare training and test sets
X = np.vstack((x1, x2, x3)).T
X_train = X[:3, :]
X_test = X[3:, :]
y_train = y1[:3]
y_test = y1[3:]

# Try these parameters in the widget
print(np.linalg.inv(X_train).dot(y1[:3]))

Discuss: how optimal is this model? What is the source of error?

Now let’s consider the case that all the inputs have an association with the model and try to minimize the error in the training set.
Discuss: how optimal is this model? What is the source of error?


Now try to re-implement the same optimization in scikit-learn?

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
print(linear_model.coef_)

Which model did you get?

What is your observation of the weight values in comparison to the true weights?

Can you propose a way to prevent the model from doing this?


You can notice that overfitting is the more common output and it is mainly associated with larger than reasonable weights overall. Introducing a penalty on the values of the weights can decrease the possibility of having large weights. This can be achieved by either L1 or L2 regularization. Let’s implement one of them in the loss function.

L1 loss function:

$$
\text{Loss} = \text{Error}(y, \hat{y}) + \lambda \sum_{i=1}^{N} \left| w_i \right|
$$


L2 loss function:

$$
\text{Loss} = \text{Error}(y, \hat{y}) + \lambda \sum_{i=1}^{N} w_i^2
$$


Where λ controls the amount of regularization.

In [ ]:
linear_model = Ridge()
linear_model.fit(X_train, y_train)
print(linear_model.coef_)

What do you observe?

##### Logistic Regression using sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
x1 = np.random.normal(size=1000)
y1 = 1.5 * x1 + np.random.normal(scale=0.5, size=1000)
y1_binary = (y1 > 0.0).astype(float)
x1 = np.expand_dims(x1, 1)
linear_model = LogisticRegression()
linear_model.fit(x1, y1_binary.squeeze())

### Explore data
Now let’s look at real brain data. We will be using data from the paper (horikawa et al paper). The dataset contains a large amount of fMRI data of images shown to subjects. Some of those images are repeatedly shown to the subject while others are shown once but different images having the same main objects (note: those images are not necessarily the ones in the dataset):

Similar: Gold fish category

<img src="https://drive.google.com/uc?id=16oO0d0TH-Bk46ThVhcXfwFi8Mkbh1sHu" alt="drawing" height="110"/> <img src="https://drive.google.com/uc?id=16oO0d0TH-Bk46ThVhcXfwFi8Mkbh1sHu" alt="drawing" height="110"/> <img src="https://drive.google.com/uc?id=16oO0d0TH-Bk46ThVhcXfwFi8Mkbh1sHu" alt="drawing" height="110"/>

Different: Cat category

<img src="https://drive.google.com/uc?id=1vSehKdq7Tl_gsUxc4xLOuSXYxF-VjJkd" alt="drawing" height="100"/> <img src="https://drive.google.com/uc?id=1FhEotidDDc9jnR-tWX4EMUXqZr0Avam4" alt="drawing" height="100"/> <img src="https://drive.google.com/uc?id=1hnbpYWpkymEOi_FWb5XxLq2pG4j53TAc" alt="drawing" height="100"/>

We will look at the data from one of the subjects (there is actually more in this dataset some of which will be explained later in part 2 and some we won’t use but can be part of your project 😉). We will be using the Brain Decoding Toolbox library for loading the data since it makes dealing with the data very easy (Also the dataset is available to be loaded directly into this format).

In [ ]:
# Get data of images from the GOD dataset
filename = 'data.h5'
voxel_data = bdpy.BData(filename)
voxel_data.show_metadata()

You now have brain data from different regions in the visual cortex corresponding to subjects watching visual stimuli. Can you build a classifier to know which image the subject is watching? We made the following function to help you extract data belonging to two different images so you can build a binary classifier.

In [ ]:
# data extraction functions
def extract_data(roi, image_type='similar', number_of_categories=2):
  filename = 'data.h5'
  data = bdpy.BData(filename)
  voxel_data = data.select(rois[roi])
  data_type = data.select('DataType')
  data_labels = data.select('stimulus_id')
  if image_type == 'similar':
    data_filter = (data_type == 2).flatten()
    voxel_data = voxel_data[data_filter, :]
    data_labels = data_labels[data_filter]
  elif image_type == 'different':
    data_filter = (data_type == 1).flatten()
    voxel_data = voxel_data[data_filter, :]
    data_labels = data_labels[data_filter]
  elif image_type == 'imagery':
    data_filter = (data_type == 3).flatten()
    voxel_data = voxel_data[data_filter, :]
    data_labels = data_labels[data_filter]
  data_labels = np.floor(data_labels).astype(int)

  unique_data_labels = np.unique(data_labels)
  unique_data_labels_filtered = unique_data_labels[:number_of_categories]
  data_label_filter = [True if label in list(unique_data_labels_filtered) else False for label in data_labels]

  voxel_data = voxel_data[data_label_filter, :]
  data_labels = data_labels[data_label_filter, :]


  return voxel_data, data_labels


In [ ]:
X, y = extract_data('LOC', image_type='similar')
print(X.shape)
print(y.shape)

### Multivariate Pattern Analysis (MVPA)

Now that you have seen how the data is structured, attempt to build a classifier and check its accuracy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model = LogisticRegression(penalty='l2')
# do a training test split

# train model

# calculate accuracy
print()


In [ ]:
# Exercise: can you visualize the decoding accuracy across different regions of interest?
roi_list = ['V1', 'V2', 'V3', 'V4', 'LOC', 'FFA', 'PPA']
accuracy = pd.DataFrame(index=roi_list, columns=['accuracy'])
for roi in roi_list:
  X, y = extract_data(roi, image_type='similar')
  
  y_pred = ...
  accuracy.loc[roi, 'accuracy'] = accuracy_score(y_pred, y_test)

accuracy.plot(kind='bar')

In [ ]:
# Exercise: can you repeat the above step but with different images from the same category
# hint, to extract different images per category, change the image_type in the extract_data function to 'different' as follows
# extract_data(ROI, image_type='different')
roi_list = ['V1', 'V2', 'V3', 'V4', 'LOC', 'FFA', 'PPA']
accuracy = pd.DataFrame(index=roi_list, columns=['accuracy'])
for roi in roi_list:
  X, y = extract_data(roi, image_type='different')
  
  y_pred = ...
  accuracy.loc[roi, 'accuracy'] = accuracy_score(y_pred, y_test)

accuracy.plot(kind='bar')

#### Think
- What does the difference in decoding accuracy tell you about the encoding of stimuli in the brain?
- Why is the decoding accuracy profile different when the images within the same category are different?
- What kind of experiments or models can you think of using decoding techniques?

## Motor decoding dataset

Can decode hand shapes from this dataset?
http://brainliner.jp/data/brainliner/Hand_shape_decoding_Rock_Papers


# References

- Neuromatch Academy Computational Neuroscience Course
- Kamitani, Yukiyasu, and Frank Tong. "Decoding the visual and subjective contents of the human brain." Nature neuroscience 8.5 (2005): 679-685.
- Horikawa, Tomoyasu, and Yukiyasu Kamitani. "Generic decoding of seen and imagined objects using hierarchical visual features." Nature communications 8.1 (2017): 1-15.
- Shen, Guohua, et al. "Deep image reconstruction from human brain activity." PLoS computational biology 15.1 (2019): e1006633.